<a href="https://colab.research.google.com/github/changedi/DPpro/blob/master/OpenaiAssistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai==1.2.3
!pip install python-dotenv

In [2]:
!pip show openai | grep Version


Version: 1.2.3


In [5]:
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

## 一步一步实现Assistant

In [6]:
from openai import OpenAI

client = OpenAI()

assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-4-1106-preview",
)
show_json(assistant)

{'id': 'asst_SZSgchaJWvsjPfsoMzjunDdm',
 'created_at': 1700558135,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': []}

create threads

In [7]:
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_Q9tQZHrPPExMzPzdVDEZBfSg',
 'created_at': 1700558168,
 'metadata': {},
 'object': 'thread'}

add messages to thread

In [8]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)
show_json(message)

{'id': 'msg_t9WlvsWROzFaUXm4svcIzN0u',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
   'type': 'text'}],
 'created_at': 1700558197,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_Q9tQZHrPPExMzPzdVDEZBfSg'}

To get a completion from an Assistant for a given Thread, we must create a Run. Creating a Run will indicate to an Assistant it should look at the messages in the Thread and take action: either by adding a single response, or using tools.

In [9]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
show_json(run)

{'id': 'run_JzAJYSTLqb9oTRbhaGa3HHQ4',
 'assistant_id': 'asst_SZSgchaJWvsjPfsoMzjunDdm',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1700558256,
 'expires_at': 1700558856,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_Q9tQZHrPPExMzPzdVDEZBfSg',
 'tools': []}

Unlike creating a completion in the Chat Completions API, creating a Run is an asynchronous operation. It will return immediately with the Run's metadata, which includes a status that will initially be set to queued. The status will be updated as the Assistant performs operations (like using tools and adding messages).

In [10]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [11]:
run = wait_on_run(run, thread)
show_json(run)

{'id': 'run_JzAJYSTLqb9oTRbhaGa3HHQ4',
 'assistant_id': 'asst_SZSgchaJWvsjPfsoMzjunDdm',
 'cancelled_at': None,
 'completed_at': 1700558259,
 'created_at': 1700558256,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1700558256,
 'status': 'completed',
 'thread_id': 'thread_Q9tQZHrPPExMzPzdVDEZBfSg',
 'tools': []}

Now that the Run has completed, we can list the Messages in the Thread to see what got added by the Assistant.



In [12]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_hS19OU83c1GvondJvS0t57Qd',
   'assistant_id': 'asst_SZSgchaJWvsjPfsoMzjunDdm',
   'content': [{'text': {'annotations': [],
      'value': 'Sure, subtract 11 from both sides to get `3x = 3`, then divide both sides by 3 to solve for `x`, which gives you `x = 1`.'},
     'type': 'text'}],
   'created_at': 1700558257,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_JzAJYSTLqb9oTRbhaGa3HHQ4',
   'thread_id': 'thread_Q9tQZHrPPExMzPzdVDEZBfSg'},
  {'id': 'msg_t9WlvsWROzFaUXm4svcIzN0u',
   'assistant_id': None,
   'content': [{'text': {'annotations': [],
      'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
     'type': 'text'}],
   'created_at': 1700558197,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': None,
   'thread_id': 'thread_Q9tQZHrPPExMzPzdVDEZBfSg'}],
 'object': 'list',
 'first_id': 'msg_hS19OU83c1GvondJvS0t57Qd',
 'las

Let's ask our Assistant to explain the result a bit further!

In [13]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="Could you explain this to me?"
)

# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
)
show_json(messages)

{'data': [{'id': 'msg_Jmu3FmLXptptlyY1Eiw5Ab53',
   'assistant_id': 'asst_SZSgchaJWvsjPfsoMzjunDdm',
   'content': [{'text': {'annotations': [],
      'value': "Certainly! Here's a step-by-step explanation:\n\n1. You start with the equation: 3x + 11 = 14.\n2. Subtract 11 from both sides to isolate the term with the variable: 3x + 11 - 11 = 14 - 11, which simplifies to 3x = 3.\n3. Divide both sides by 3 to solve for x: (3x)/3 = 3/3, which simplifies to x = 1. \n\nNow you have the solution: x = 1."},
     'type': 'text'}],
   'created_at': 1700558468,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_ljCb3IWrkJQhwkhw6DrpxBif',
   'thread_id': 'thread_Q9tQZHrPPExMzPzdVDEZBfSg'}],
 'object': 'list',
 'first_id': 'msg_Jmu3FmLXptptlyY1Eiw5Ab53',
 'last_id': 'msg_Jmu3FmLXptptlyY1Eiw5Ab53',
 'has_more': False}

## 例子
整理一个集合全部功能的例子，如果需要，前置步骤得加一个Assistant的创建

In [14]:
from openai import OpenAI

MATH_ASSISTANT_ID = assistant.id  # or a hard-coded ID like "asst-..."

client = OpenAI()

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(MATH_ASSISTANT_ID, thread, user_input)
    return thread, run

In [15]:
# Emulating concurrent user requests
thread1, run1 = create_thread_and_run(
    "I need to solve the equation `3x + 11 = 14`. Can you help me?"
)
thread2, run2 = create_thread_and_run("Could you explain linear algebra to me?")
thread3, run3 = create_thread_and_run("I don't like math. What can I do?")

# Now all Runs are executing...

In [16]:
import time

# Pretty printing helper
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()


# Waiting in a loop
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run


# Wait for Run 1
run1 = wait_on_run(run1, thread1)
pretty_print(get_response(thread1))

# Wait for Run 2
run2 = wait_on_run(run2, thread2)
pretty_print(get_response(thread2))

# Wait for Run 3
run3 = wait_on_run(run3, thread3)
pretty_print(get_response(thread3))

# Thank our assistant on Thread 3 :)
run4 = submit_message(MATH_ASSISTANT_ID, thread3, "Thank you!")
run4 = wait_on_run(run4, thread3)
pretty_print(get_response(thread3))

# Messages
user: I need to solve the equation `3x + 11 = 14`. Can you help me?
assistant: Yes, subtract 11 from both sides to get `3x = 3`, and then divide by 3 to find `x = 1`.

# Messages
user: Could you explain linear algebra to me?
assistant: Linear algebra is the branch of mathematics that deals with vectors, vector spaces, linear transformations, and systems of linear equations, emphasizing the properties and operations in multidimensional spaces.

# Messages
user: I don't like math. What can I do?
assistant: Try to find aspects of math that relate to your interests, or look for fun and practical applications to make it more engaging and relevant to your life.

# Messages
user: I don't like math. What can I do?
assistant: Try to find aspects of math that relate to your interests, or look for fun and practical applications to make it more engaging and relevant to your life.
user: Thank you!
assistant: You're welcome! If you need any more help or have other questions, feel free to 

## 使用工具

### 先看看Code Interpreter


In [17]:
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[{"type": "code_interpreter"}],
)
show_json(assistant)

{'id': 'asst_SZSgchaJWvsjPfsoMzjunDdm',
 'created_at': 1700558135,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}]}

In [18]:
thread, run = create_thread_and_run(
    "Generate the first 20 fibbonaci numbers with code."
)
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: Generate the first 20 fibbonaci numbers with code.
assistant: The first 20 Fibonacci numbers are: 
0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181.



A Run is composed of one or more Steps. Like a Run, each Step has a status that you can query. This is useful for surfacing the progress of a Step to a user (e.g. a spinner while the Assistant is writing code or performing retrieval).



In [19]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id, run_id=run.id, order="asc"
)

In [20]:
for step in run_steps.data:
    step_details = step.step_details
    print(json.dumps(show_json(step_details), indent=4))

{'tool_calls': [{'id': 'call_rtqPePzfWGi5XtI8RmRaVtx0',
   'code_interpreter': {'input': 'def fibonacci_sequence(n):\n    sequence = [0, 1]\n    while len(sequence) < n:\n        sequence.append(sequence[-1] + sequence[-2])\n    return sequence\n\nfirst_20_fibonacci_numbers = fibonacci_sequence(20)\nfirst_20_fibonacci_numbers',
    'outputs': [{'logs': '[0,\n 1,\n 1,\n 2,\n 3,\n 5,\n 8,\n 13,\n 21,\n 34,\n 55,\n 89,\n 144,\n 233,\n 377,\n 610,\n 987,\n 1597,\n 2584,\n 4181]',
      'type': 'logs'}]},
   'type': 'code_interpreter'}],
 'type': 'tool_calls'}

null


{'message_creation': {'message_id': 'msg_YXyTfjGIlmYh0EgEM3IzMJZ0'},
 'type': 'message_creation'}

null


### 接下来，检索
Another powerful tool in the Assistants API is Retrieval: the ability to upload files that the Assistant will use as a knowledge base when answering questions. This can also be enabled from the Dashboard or the API, where we can upload files we want to be used.

In [ ]:
!wget https://github.com/openai/openai-cookbook/blob/feef1bf3982e15ad180e17732525ddbadaf2b670/examples/data/language_models_are_unsupervised_multitask_learners.pdf

In [22]:
# Upload the file
file = client.files.create(
    file=open(
        "language_models_are_unsupervised_multitask_learners.pdf",
        "rb",
    ),
    purpose="assistants",
)

# Update Assistant
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[{"type": "code_interpreter"}, {"type": "retrieval"}],
    file_ids=[file.id],
)
show_json(assistant)

{'id': 'asst_SZSgchaJWvsjPfsoMzjunDdm',
 'created_at': 1700558135,
 'description': None,
 'file_ids': ['file-L8SbCs76d5eR9hiiAUkqm1RF'],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}, {'type': 'retrieval'}]}

In [25]:
thread, run = create_thread_and_run(
    "What are some cool math concepts behind this ML paper pdf? Explain in two sentences."
)
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: What are some cool math concepts behind this ML paper pdf? Explain in two sentences.
assistant: I am unable to provide the content requested because the PDF's format is not supported for text extraction in the current toolset. If you could specify a particular section or concept, or convert the PDF to a text-based format, I may be able to assist further.

